# Import

In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, regularizers, evaluators, trainers, datasets, distributions

# DataSet

In [2]:
dataset = datasets.ML20m()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)

# Evaluator

In [3]:
ks = [5, 10, 50]
recall = evaluators.RecallEvaluator(test_set, ks)

# Sampler

In [4]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

# Model

In [6]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 50
valid_per_epoch = 10
n_item_sample = 30
n_user_sample = 30
no_progressbar = True

# distributiuons
# dist1 = distributions.Empirical()
dist2 = distributions.Gaussian()
dist3 = distributions.Gamma()

# models
model0 = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
# model1 = models.MutualProximityCML(n_user, n_item, dist1, n_dim, n_item_sample, n_user_sample).to(device)
model2 = models.MutualProximityCML(n_user, n_item, dist2, n_dim, n_item_sample, n_user_sample).to(device)
model3 = models.MutualProximityCML(n_user, n_item, dist3, n_dim, n_item_sample, n_user_sample).to(device)

# learning late optimizer
optimizer0 = optim.Adam(model0.parameters(), lr=lr)
# optimizer1 = optim.Adam(model1.parameters(), lr=lr)
optimizer2 = optim.Adam(model2.parameters(), lr=lr)
optimizer3 = optim.Adam(model3.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer0 = trainers.BaseTrainer(model0, optimizer0, criterion, sampler, no_progressbar)
# trainer1 = trainers.BaseTrainer(model1, optimizer1, criterion, sampler, no_progressbar)
trainer2 = trainers.BaseTrainer(model2, optimizer2, criterion, sampler, no_progressbar)
trainer3 = trainers.BaseTrainer(model3, optimizer3, criterion, sampler, no_progressbar)

# Base CML

In [7]:
trainer0.fit(n_batch, n_epoch, recall, valid_per_epoch)

KeyboardInterrupt: 

In [ ]:
# CML
trainer0.valid_scores

# Gaussian

In [ ]:
trainer2.fit(n_batch, n_epoch, recall, valid_per_epoch)

In [ ]:
trainer2.valid_scores

# Gamma

In [ ]:
trainer3.fit(n_batch, n_epoch, recall, valid_per_epoch)

In [ ]:
trainer3.valid_scores